## This notebook demonstrates how to manage permissions to your projects

In [ ]:
from adam import ConfigManager
from adam import Permission
from adam import Service
import os

This sets up a Service which uses the given token and URL to provide authorized access through the server through several wrapped modules. It also creates a project for you to work in that will be used for the rest of the notebook. Be sure to run service.teardown() when finished. 

If you don't have a config, see config_demo to get one.

In [ ]:
# Reads your config from a file in current directory. For instructions on setting this up, see config_demo notebook.
config = ConfigManager(os.getcwd() + '/config.json').get_config()
service = Service(config)
service.setup()

In [ ]:
team_project = service.new_working_project()

Let's suppose that you're managing ADAM resources for a team of people. You're working on a couple different things, codenamed CatOrbits and DogOrbits. You've got some teammates, agatha@orbitinganimals.com, belinda@orbitinganimals.com, and calliope@orbitinganimals.com. Agatha and Belinda work on CatOrbits and Belinda works with Calliope on DogOrbits. It would be a good idea to manage the propagations and such related to CatOrbits and DogOrbits separately, so you create a project for each of them within your working project.

In [ ]:
projects = service.get_projects_module()
cat_orbits_project = projects.new_project(
    team_project.get_uuid(), "CatOrbits", "Project to use for CatOrbits-related stuff")
dog_orbits_project = projects.new_project(
    team_project.get_uuid(), "DogOrbits", "Project to use for DogOrbits-related stuff")

Now that you have a project for each major thing your team is working on, let's give your teammates access to the projects. Let's suppose that you want only the team members working on each project to be able to run propagations in that project, but you want the whole team to be able to grab the results of those propagations. Running propagations in a project requires WRITE permission. Getting the propagation results requires READ permissions.

To do this, you make a team group and add everybody to it. Then you give the group READ permission to your team project. Since access to a project provides the same access to all its child objects, this gives your team READ access to cat_orbits_project, dog_orbits_project, and any other child projects you add in the future.

Then you directly give Agatha and Belinda WRITE access to CatOrbits and Belinda and Calliope WRITE access to DogOrbits. You could use groups for this too, but this is a demo, so we're going to do things in ALL the ways.

Finally, suppose you are really not all that involved in the day-to-day of CatOrbits and DogOrbits and you'd really like Belinda to be able to manage permissions, etc on those projects, so you decide to give Belinda ADMIN access to all objects.

In [ ]:
groups = service.get_groups_module()
permissions = service.get_permissions_module()

team_group = groups.new_group("OrbitingAnimalsGroup", "Group containing every member of the OrbitingAnimals team")
groups.add_user_to_group("agatha@orbitinganimals.com", team_group.get_uuid())
groups.add_user_to_group("belinda@orbitinganimals.com", team_group.get_uuid())
groups.add_user_to_group("calliope@orbitinganimals.com", team_group.get_uuid())

In [ ]:
permissions.grant_group_permission(
    team_group.get_uuid(), Permission('READ', 'PROJECT', team_project.get_uuid()))

In [ ]:
permissions.grant_user_permission(
    "agatha@orbitinganimals.com", Permission('WRITE', 'PROJECT', cat_orbits_project.get_uuid()))
permissions.grant_user_permission(
    "belinda@orbitinganimals.com", Permission('WRITE', 'PROJECT', cat_orbits_project.get_uuid()))

In [ ]:
permissions.grant_user_permission(
    "belinda@orbitinganimals.com", Permission('WRITE', 'PROJECT', dog_orbits_project.get_uuid()))
permissions.grant_user_permission(
    "calliope@orbitinganimals.com", Permission('WRITE', 'PROJECT', dog_orbits_project.get_uuid()))

In [ ]:
permissions.grant_user_permission(
    "belinda@orbitinganimals.com", Permission('ADMIN', 'PROJECT', team_project.get_uuid()))
permissions.grant_user_permission(
    "belinda@orbitinganimals.com", Permission('ADMIN', 'GROUP', team_group.get_uuid()))

All right, let's look at what you've set up. Listing permissions by target objects rather than permitted entity or for users other than yourself isn't currently supported, but may be in the near future.

In [ ]:
groups.print_group_members(team_group.get_uuid())
permissions.print_group_permissions(team_group.get_uuid())

Now you are set up so that your team members have workspaces in which to run propagations! So now suppose that Agatha wants to run some experimental propagations of Maine Coon orbits. She can create a project within cat_orbits_project because she has WRITE access to it. When she creates the project, she'll automatically be given ADMIN rights as the creator (and also her WRITE rights will apply from cat_orbits_project). So she will be able to create Batch objects within maine_coon_orbits by assigning its uuid as the project_uuid in each Batch. See the other demo notebooks, e.g. Test_Single_Batch_Run, to see how to do this. 

In [ ]:
# Clean up the project and groups.
groups.delete_group(team_group.get_uuid())
projects.delete_project(cat_orbits_project.get_uuid())
projects.delete_project(dog_orbits_project.get_uuid())
service.teardown()